In [28]:
from selenium import webdriver
from time import sleep
ch_driver = webdriver.Chrome('C:/Users/beave/AppData/Roaming/Microsoft/Windows/Start Menu/Programs/Python 3.7/chromedriver.exe')

ch_driver.implicitly_wait(5) ##이건 어디에 둬야하나. 한번 두면 계속 적용되는건가

yelp_url = 'https://www.yelp.com/'
ch_driver.get(yelp_url)

In [29]:
search_input = ch_driver.find_element_by_id('find_desc')
search_input.send_keys('restaurants')
city_input = ch_driver.find_element_by_id('dropperText_Mast')
city_input.click()
city_input.send_keys('los angeles')

search_button = ch_driver.find_element_by_id('header-search-submit')
search_button.click()

In [39]:
#일단 광고 게시물 제거해서 list로 만들기
def get_restaurant_list():
    result_list = []
   
   
    r_path = '//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li/div/div/div/div/div[2]/div[1]/div[1]/div[1]'
    restaurants = ch_driver.find_elements_by_xpath(r_path)
    for i,item in enumerate(restaurants):
        title = item.find_element_by_tag_name('h3').text
        name = item.find_element_by_tag_name('a').text
        link = item.find_element_by_tag_name('a').get_attribute('href')

        #get rating
        rating_elem = ch_driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li[' +
                                                      str(i+1)+']/div/div/div/div/div[2]/div[1]/div[1]/div[1]/div[2]/span/div')
        rating = rating_elem.get_attribute('aria-label')
        #print(rating)

        #get number of reviews
        review_elem = ch_driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li['
                                                      + str(i+1)+']/div/div/div/div/div[2]/div[1]/div[1]/div[1]/div[3]/span')
        review_num = review_elem.text

        #get price index and category
        try:
            pc_elem = ch_driver.find_elements_by_xpath('//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li[' +
                                                       str(i+1) + ']/div/div/div/div/div[2]/div[1]/div[1]/div[1]/div[4]/span')
            price = pc_elem[0].text
            category = pc_elem[1].text
        except IndexError:
            pc_elem = ch_driver.find_element_by_xpath('//*[@id="wrap"]/div[3]/div[2]/div[2]/div/div[1]/div[1]/div/ul/li[' +
                                                       str(i+1) + ']/div/div/div/div/div[2]/div[1]/div[1]/div[1]/div[4]/span')
            price = "N/A"
            category = pc_elem.text
        
        #전체 var 받은뒤 title로 광고 게시물 거르기
        if title[0].isdigit() == False:
            continue
            
        result_list.append([name, link, rating, review_num, price, category])
        
    return result_list

In [40]:
get_restaurant_list()

[['Howlin’ Ray’s',
  'https://www.yelp.com/biz/howlin-rays-los-angeles-3?osq=restaurants',
  '4.5 star rating',
  '4312 reviews',
  '$$',
  'Southern, Chicken Shop, American (Traditional)'],
 ['Bestia',
  'https://www.yelp.com/biz/bestia-los-angeles?osq=restaurants',
  '4.5 star rating',
  '5343 reviews',
  '$$$',
  'Italian, Cocktail Bars, Pizza'],
 ['Myungrang Hot Dog-California Market LA',
  'https://www.yelp.com/biz/myungrang-hot-dog-california-market-la-los-angeles?osq=restaurants',
  '4 star rating',
  '44 reviews',
  'N/A',
  'Hot Dogs'],
 ['Running Goose',
  'https://www.yelp.com/biz/running-goose-los-angeles?osq=restaurants',
  '4.5 star rating',
  '757 reviews',
  '$$',
  'Tapas/Small Plates, Gastropubs, Latin American'],
 ['RockSugar Southeast Asian Kitchen',
  'https://www.yelp.com/biz/rocksugar-southeast-asian-kitchen-los-angeles?osq=restaurants',
  '4 star rating',
  '2177 reviews',
  '$$',
  'Indonesian, Vietnamese, Malaysian'],
 ['Daikokuya Little Tokyo',
  'https://www

In [41]:
#ranking이 key, list결과물을 value로 하는 dict 만들기 
def get_restaurants():
    result = {}
    result_list = get_restaurant_list()
    
    for i in range(30):
       
        result[i+1] = (result_list[i])
        
    return result
        

In [42]:
get_restaurants()

{1: ['Howlin’ Ray’s',
  'https://www.yelp.com/biz/howlin-rays-los-angeles-3?osq=restaurants',
  '4.5 star rating',
  '4312 reviews',
  '$$',
  'Southern, Chicken Shop, American (Traditional)'],
 2: ['Bestia',
  'https://www.yelp.com/biz/bestia-los-angeles?osq=restaurants',
  '4.5 star rating',
  '5343 reviews',
  '$$$',
  'Italian, Cocktail Bars, Pizza'],
 3: ['Myungrang Hot Dog-California Market LA',
  'https://www.yelp.com/biz/myungrang-hot-dog-california-market-la-los-angeles?osq=restaurants',
  '4 star rating',
  '44 reviews',
  'N/A',
  'Hot Dogs'],
 4: ['Running Goose',
  'https://www.yelp.com/biz/running-goose-los-angeles?osq=restaurants',
  '4.5 star rating',
  '757 reviews',
  '$$',
  'Tapas/Small Plates, Gastropubs, Latin American'],
 5: ['RockSugar Southeast Asian Kitchen',
  'https://www.yelp.com/biz/rocksugar-southeast-asian-kitchen-los-angeles?osq=restaurants',
  '4 star rating',
  '2177 reviews',
  '$$',
  'Indonesian, Vietnamese, Malaysian'],
 6: ['Daikokuya Little Toky

In [43]:
all_result = {}
all_result['Los Angeles'] = get_restaurants()

In [44]:
sleep(5)

city_list = ['San Francisco', 'New York']

for city in city_list:
    #다른 도시로 이동
    another_city = ch_driver.find_element_by_id('search_location')
    another_city.click()
    another_city.send_keys(city)
    another_button = ch_driver.find_element_by_xpath('//*[@id="header_find_form"]/div/div[2]/div/div[2]/button')
    another_button.click()
    
    #그 도시 스크래핑한 내용 all_result에 담기
    all_result[city] = get_restaurants()
    sleep(5)

In [45]:
all_result

{'Los Angeles': {1: ['Howlin’ Ray’s',
   'https://www.yelp.com/biz/howlin-rays-los-angeles-3?osq=restaurants',
   '4.5 star rating',
   '4312 reviews',
   '$$',
   'Southern, Chicken Shop, American (Traditional)'],
  2: ['Bestia',
   'https://www.yelp.com/biz/bestia-los-angeles?osq=restaurants',
   '4.5 star rating',
   '5343 reviews',
   '$$$',
   'Italian, Cocktail Bars, Pizza'],
  3: ['Myungrang Hot Dog-California Market LA',
   'https://www.yelp.com/biz/myungrang-hot-dog-california-market-la-los-angeles?osq=restaurants',
   '4 star rating',
   '44 reviews',
   'N/A',
   'Hot Dogs'],
  4: ['Running Goose',
   'https://www.yelp.com/biz/running-goose-los-angeles?osq=restaurants',
   '4.5 star rating',
   '757 reviews',
   '$$',
   'Tapas/Small Plates, Gastropubs, Latin American'],
  5: ['RockSugar Southeast Asian Kitchen',
   'https://www.yelp.com/biz/rocksugar-southeast-asian-kitchen-los-angeles?osq=restaurants',
   '4 star rating',
   '2177 reviews',
   '$$',
   'Indonesian, Vietnam